In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')
# 참고 사이트 : https://m.blog.naver.com/anne9/221798711142
#  내 드라이브 위치 : '/content/gdrive/My Drive/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import sys
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [3]:
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/BERT_Preprocessing')

In [4]:
import time
import nltk
import pandas as pd
import re
import nltk
from nltk.corpus import stopl have"
from contractions_sentiment import CONTRACION_MAP 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm # 상태 표시줄
from multiprocessing import Pool # 병렬 처리

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no') # 불용어 list에서 감정분석에 영향이 큰 no, not 제거
stopword_list.remove('not')

stemmer = nltk.stem.SnowballStemmer('english') # Stemming(어간 추출) : 형태소적 형태를 간소화
lemma = WordNetLemmatizer() # Lemmatization(표제어 추출) : 단어의 기본 사전형 추출

print("rocks :", lemma.lemmatize("rocks")) 
print("corpora :", lemma.lemmatize("corpora")) 
print("was :", lemma.lemmatize("was")) 
print("were :", lemma.lemmatize("were")) 
print("is :", lemma.lemmatize("is"))
print("are :", lemma.lemmatize("are"))

print(lemma.lemmatize("was", 'v'))
print(lemma.lemmatize("are", 'v')) # lemma.lemmatize document 찾아보기

rocks : rock
corpora : corpus
was : wa
were : were
is : is
are : are
be
be


In [10]:
def get_wordnet_pos(word):
    #print(nltk.pos_tag([word])[0])
    #print(nltk.pos_tag([word])[0][1])
    #print(nltk.pos_tag([word])[0][1][0])
    tag = nltk.pos_tag([word])[0][1][0].upper() #pos_tag : Part-Of-Speech Tag, 품사 태깅
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), # contraction_mapping.keys : 바꾸려는 축약어
                                      flags=re.IGNORECASE|re.DOTALL)
    # re.compile : ''안에 정규표현식 수행 / flags ~ | 로 묶어서 수행할 정규표현식에 옵션 추가
    # re.IGNORECASE : 대소문자 구분하지 않음
    # re.DOTALL : 정규표현식에서 . 의 의미로 '모든' 문자
    def expand_match(contraction):
        match = contraction.group(0) # group(0) : group0에 매칭된 단어 반환 // group 0, group 1 까지 있음
        #match2 = contraction.group(1) # group0이랑 무슨 차이?
        #print('match : ',match) # 위에서 정의한 축약어 패턴과 매치하는 부분 찾음
        #print('match2 : ', match2)
        first_char = match[0] # 원본 첫번째 문자 따로 빼놓음 -> 아래에서 붙일지 말지 때문에
        # match 값 가져와서 mapping 되면 그 값 가져오고
        # 안 맞으면 소문자화해서 다시 mapping
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower()) 
        expanded_contraction = first_char+expanded_contraction[1:] # 원본 첫번째 문자가 대문자이면 유지하고 붙여서 return
        #print('expanded contraction : ',expanded_contraction) # 축약어 풀어서 변환
        return expanded_contraction
    
    # re.sub(pattern, replacement, string)
    expanded_text = contractions_pattern.sub(expand_match, text)
    # sub : text에서 contractions_pattern에 맞는 것을 찾앚을 시 expand_match 함수에 맞게 바꿈
    #expanded_text = re.sub("'", "", expanded_text) # sub : expanded_text에서 ' 을 공백으로 바꿈 # 이 코드 없어도 위에서 변환 잘 돼서 주석처리
    return expanded_text

expand_contractions("Y'all Can't expand contractions I'd think")
# contraction 목록에 Y'all 은 없고 y'all 은 있음

'You all Can not expand contractions I would think'

In [12]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3', 'lol', 'LOL'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])


#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons / unicode U0001F600: 웃는 모양 
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [13]:
def ingest():
    data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/BERT_Preprocessing/tweets.csv', encoding='latin-1') # 파일 저장소 디렉토리
    data.columns=["Sentiment","ItemID","Date","Blank","SentimentSource","SentimentText"]
    data.drop(['ItemID', 'SentimentSource'], axis=1, inplace=True)
    data = data[data.Sentiment.isnull() == False] # null이 아닌 값만 가져옴
    data['Sentiment'] = data['Sentiment'].map( {4:1, 0:0}) #Converting 4 to 1 # 원본 데이터에 4->1로, 0->0으로 바꿔줌
    data = data[data['SentimentText'].isnull() == False] # null이 아닌 값만 가져옴
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape  )  
    return data

data = ingest()

dataset loaded with shape (1599999, 4)


In [14]:
data.head()

,Sentiment,Date,Blank,SentimentText
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire
3,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,"@nationwideclass no, it's not behaving at all...."
4,0,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,@Kwesidei not the whole crew


In [15]:
data[data['Sentiment']==1].head()

,Sentiment,Date,Blank,SentimentText
799999,1,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,I LOVE @Health4UandPets u guys r the best!!
800000,1,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,im meeting up with one of my besties tonight! ...
800001,1,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,1,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,Being sick can be really cheap when it hurts t...
800003,1,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,@LovesBrooklyn2 he has that effect on everyone


In [16]:
# 원본 데이터가 160만 개에서 한 개가 부족해서 데이터 한 개 추가
df = pd.DataFrame(columns=['Sentiment', 'Date', 'Blank', 'SentimentText'])
df.loc[0] = [0,'Mon Apr 06 22:20:20 PDT 2009','NO_QUERY',
             "@smarrison i would've been the first, but i didn't have a gun. not really though, zac snyder's just a doucheclown."]
df2 = data.append(df , ignore_index=True)
len(df2)

1600000

In [17]:
data = df2

In [18]:
def data_text_cleaning_lemma(data):
    tweet = data.lower().strip() # 소문자화
    tweet = tweet.replace("’","'") # 작은 따움표 표기법 바꾸기 
    #print(tweet)
    
    if tweet.startswith('=@'): # 일부 예외적 데이터 수정
        tweet = tweet[1:]
    elif tweet.startswith('rt @'):
        tweet = tweet[3:]
    else:
        tweet = tweet 
        
    data_split = tokenizer.tokenize(tweet) # 단어 토큰화 
    no_contractions = [expand_contractions(x) for x in data_split] # 축약어 처리
    data_split2 = []

    for s in range(len(no_contractions)):
        partial = tokenizer.tokenize(no_contractions[s]) # 처리한 축약어 토큰화
        for x in range(len(partial)):
            data_split2.append(partial[x]) # 처리한 토큰 추가

    # @, #, http 로 시작하는 토큰 제거 
    trim_data = [x for x in data_split2 if not x.startswith('@') | x.startswith('=@')| x.startswith('http') | x.startswith('#')]
    # data_split2에 있는 x가 @, =@, http, # 으로 시작 하지 않으면 trim_data에 추가
    # 이모티콘과 이모지 토큰 제거 
    trim_data = [w for w in trim_data if not w in emoticons]
    # trim_data에 있는 w가 emoticons에 없으면 trim_data에 추가 
    trim_data2 = []

    for s in range(len(trim_data)):
        #print(data_split[s])
        trim_data2.append(emoji_pattern.sub(r'', trim_data[s])) # trim_data의 s가 emoji_pattern에 맞으면 공백으로 바꾸어 trim_data2에 추가
    
    trim_data3 = []

    # trim_data2에서 길이가 0이 아닌 것들을 trim_data3에 추가
    for s in range(len(trim_data2)):
        if len(trim_data2[s]) == 0:
            pass
        else:
            trim_data3.append(trim_data2[s])

    sample = ' '.join(trim_data3).strip() # join: list를 ' '으로 묶어 string으로 만듦 -> strip : 앞 뒤 공백 문자 없엠
    sample = tokenizer.tokenize(sample)

    # 영문자 이외 문자는 공백으로 변환
    only_english = list(filter(lambda t: t.isalpha(), sample))
    listToStr = ' '.join([str(elem) for elem in only_english]) 
    
    #무의미하게 반복된 철자 수정
    sentence = ''.join(['' if i > 1 and e == listToStr[i-2] and e == listToStr[i-1] else e for i,e in enumerate(listToStr)])
    sentence = re.sub(r'\s+', ' ', sentence)
        
    #표제어 추출
    sentence_lemma = [lemma.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
    
    # 불용어 제거 
    tokens_without_stopwords = [x for x in sentence_lemma if x not in stopword_list]
    
    if len(tokens_without_stopwords) == 0:
        tokens_without_stopwords = 'NC'
    else:
        pass

    return tokens_without_stopwords

In [19]:
tokenizer = TweetTokenizer()

tqdm.pandas(desc="progress-bar")

def use_multiprocess(func, iter, workers):
    pool = Pool(processes=workers)
    result = pool.map(func, iter)
    pool.close()
    return result

In [20]:
if __name__ == '__main__':
    start_time = time.time()
    
    data['TrimText_lemma'] = use_multiprocess(data_text_cleaning_lemma, data['SentimentText'], 10)
    
    print('실행 시간 :', round((time.time() - start_time)/60, 2), 'min')

실행 시간 : 43.8 min


In [21]:
data.head()

,Sentiment,Date,Blank,SentimentText,TrimText_lemma
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,..."
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g..."
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]"
3,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,"@nationwideclass no, it's not behaving at all....","[no, not, behaving, mad, not, see]"
4,0,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,@Kwesidei not the whole crew,"[not, whole, crew]"


In [22]:
def count(data):
    return len(data)

if __name__ == '__main__':
    start_time = time.time()
    
    data['TrimText_lemma_len'] = use_multiprocess(count, data['TrimText_lemma'], 10)
    
    print('실행 시간 :', round((time.time() - start_time)/60, 2), 'min')

실행 시간 : 0.08 min


In [23]:
data.head()

,Sentiment,Date,Blank,SentimentText,TrimText_lemma,TrimText_lemma_len
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,...",12
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g...",9
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]",6
3,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,"@nationwideclass no, it's not behaving at all....","[no, not, behaving, mad, not, see]",6
4,0,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,@Kwesidei not the whole crew,"[not, whole, crew]",3


In [24]:
data['TrimText_lemma_len'].describe()
# 평균 6.98
# 표준편차 3.88
# 최소값 : 1
# median : 6
# 최대값 : 116

count    1.600000e+06
mean     6.980012e+00
std      3.881035e+00
min      1.000000e+00
25%      4.000000e+00
50%      6.000000e+00
75%      1.000000e+01
max      1.160000e+02
Name: TrimText_lemma_len, dtype: float64

In [25]:
# 토큰 26개 넘는 데이터 삭제
data2 = data[data.TrimText_lemma_len < 26]
print(len(data2))
print(len(data2[data2.Sentiment == 0]))
print(len(data2[data2.Sentiment == 1]))

1599498
799851
799647


In [26]:
data2.TrimText_lemma[0]

['upset',
 'not',
 'update',
 'facebook',
 'texting',
 'might',
 'cry',
 'result',
 'school',
 'today',
 'also',
 'blah']

In [28]:
def join_data(txt):
    txt = ' '.join(txt)
    return txt

if __name__ == '__main__':
    start_time = time.time()

    data2['TrimText_lemma_join'] = use_multiprocess(join_data, data2['TrimText_lemma'], 10)

    print('실행 시간 :', round((time.time() - start_time)/60, 2), 'min')

실행 시간 : 0.09 min


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
data2.head()

,Sentiment,Date,Blank,SentimentText,TrimText_lemma,TrimText_lemma_len,TrimText_lemma_join
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,...",12,upset not update facebook texting might cry re...
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g...",9,dive many time ball manage save rest go bound
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]",6,whole body feel itchy like fire
3,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,"@nationwideclass no, it's not behaving at all....","[no, not, behaving, mad, not, see]",6,no not behaving mad not see
4,0,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,@Kwesidei not the whole crew,"[not, whole, crew]",3,not whole crew


In [33]:
data2 = data2.rename(columns={"TrimText_lemma_join": "TrimText_lemma_len_join"})

In [34]:
print(len(data2))
print(type(data2['TrimText_lemma_len_join'][0]))

1599498
<class 'str'>


In [35]:
data2.head()

,Sentiment,Date,Blank,SentimentText,TrimText_lemma,TrimText_lemma_len,TrimText_lemma_len_join
0,0,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,is upset that he can't update his Facebook by ...,"[upset, not, update, facebook, texting, might,...",12,upset not update facebook texting might cry re...
1,0,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,@Kenichan I dived many times for the ball. Man...,"[dive, many, time, ball, manage, save, rest, g...",9,dive many time ball manage save rest go bound
2,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,my whole body feels itchy and like its on fire,"[whole, body, feel, itchy, like, fire]",6,whole body feel itchy like fire
3,0,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,"@nationwideclass no, it's not behaving at all....","[no, not, behaving, mad, not, see]",6,no not behaving mad not see
4,0,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,@Kwesidei not the whole crew,"[not, whole, crew]",3,not whole crew


In [36]:
print(data2['TrimText_lemma_len'].median())
print(data2['TrimText_lemma_len'].max())

6.0
25


In [40]:
neg = data2[data2.Sentiment == 0]
print(len(neg))
pos = data2[data2.Sentiment == 1]
print(len(pos))

799851
799647


In [41]:
train = neg[:500000].append(pos[:500000])
test = neg[500000:700000].append(pos[500000:700000])
print(len(train))
print(len(test))

1000000
400000


In [43]:
train.to_pickle("/content/gdrive/My Drive/Colab Notebooks/BERT_Preprocessing/train_data_210222.pkl")
test.to_pickle("/content/gdrive/My Drive/Colab Notebooks/BERT_Preprocessing/test_data_210222.pkl")